In [1]:
import os, glob, re
from datetime import date
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from osgeo import gdal
import json, geojson
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
import folium
from PIL import Image
import earthpy.plot as ep

import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 250

In [2]:
from IPython.display import IFrame
src = 'https://www.keene.edu/campus/maps/tool/'
IFrame(src, width=960, height=500)

In [3]:
AREA = [
    [
        139.708159,
        35.6593884   
    ],
    [
        139.7067857,
        35.4817801
    ],
    [
        139.8619676,
        35.4817801
    ],
    [
        139.8578477,
        35.6493456
    ],
    [
        139.708159,
        35.6593884   
    ]
]

In [4]:
for i in range(len(AREA)):
    if AREA[i][0] >= 0:
        AREA[i][0] = AREA[i][0]%360
    else:
        AREA[i][0] = -(abs(AREA[i][0]%360) + 360)

In [5]:
m = geojson.Polygon([AREA])

object_name = 'Tokyo_Bay'

with open(str(object_name) + '.geojson', 'w') as f:
    json.dump(m, f)
footprint_geojson = geojson_to_wkt(read_geojson(str(object_name) + '.geojson'))

In [6]:
user = 'wing787'
password = 'wing787_api'
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [7]:
m = folium.Map([(AREA[0][1]+AREA[len(AREA)-1][1])/2,(AREA[0][0]+AREA[len(AREA)-1][0])/2], zoom_start=10)

folium.GeoJson(str(object_name) + '.geojson').add_to(m)
m

In [8]:
products = api.query(footprint_geojson,
                     date = ('20211110', '20211120'), # 取得希望期間の入力
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,100)) # 被雲率（0％～100％）

In [9]:
len(products)

2

In [10]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid,geometry
d01bf938-8494-43f3-817b-3780f469064a,S2A_MSIL2A_20211113T012911_N0301_R074_T54SUE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-11-13T01:29:11.024Z, Instrument: MS...",false,2021-11-13 04:01:03,2021-11-13 01:29:11.024,2021-11-13 01:29:11.024,2021-11-13 06:25:40.856,...,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS4_20211113T040103_S2021...,S2A_OPER_MSI_L2A_TL_VGS4_20211113T040103_A0333...,S2A_MSIL2A_20211113T012911_N0301_R074_T54SUE_2...,d01bf938-8494-43f3-817b-3780f469064a,"MULTIPOLYGON (((138.80483 35.13500, 140.00972 ..."
c6135c85-ee17-4dc8-9bc9-27a68edd9890,S2B_MSIL2A_20211118T012939_N0301_R074_T54SUE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-11-18T01:29:39.024Z, Instrument: MS...",false,2021-11-18 03:43:40,2021-11-18 01:29:39.024,2021-11-18 01:29:39.024,2021-11-18 05:56:05.105,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20211118T034340_S2021...,S2B_OPER_MSI_L2A_TL_VGS4_20211118T034340_A0245...,S2B_MSIL2A_20211118T012939_N0301_R074_T54SUE_2...,c6135c85-ee17-4dc8-9bc9-27a68edd9890,"MULTIPOLYGON (((138.80483 35.13500, 140.00972 ..."


In [ ]:
# ファイルが重いため、処理時間がかかりやすい
uuid = products_gdf_sorted.iloc[0]['uuid']
product_title = products_gdf_sorted.iloc[0]['title']
api.download(uuid)